In [1]:
import boto3
import configparser
import json
import pandas as pd

# Cluster, S3, IAM configurations

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_IAM_ROLE_NAME      = config.get('IAM_ROLE', 'DWH_IAM_ROLE_NAME')

DB_NAME                = config.get('CLUSTER', 'DB_NAME')
DB_USER                = config.get('CLUSTER', 'DB_USER')
DB_PASSWORD            = config.get('CLUSTER', 'DB_PASSWORD')
DB_PORT                = config.get('CLUSTER', 'DB_PORT')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")



# Creating a iam Role

In [3]:
iam = boto3.client('iam',
                  region_name='us-west-2',
                  aws_access_key_id = KEY,
                  aws_secret_access_key = SECRET)

In [ ]:
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description='Allows Redshift clusters to call AWS services on your behalf.',
        AssumeRolePolicyDocument= json.dumps(
            {'Statement':[{'Action': 'sts:AssumeRole',
                          'Effect': 'Allow',
                          'Principal': {'Service': 'redshift.amazonaws.com'}}],
                         'Version': '2012-10-17'})
    )
except Exception as e:
    print(e)

In [4]:
print('1.2 Attaching Policy')
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                      PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
                      )['ResponseMetadata']['HTTPStatusCode']

1.2 Attaching Policy


200

In [5]:
print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.3 Get the IAM role ARN
arn:aws:iam::530010734432:role/dwhRole


# Access to S3

In [ ]:
s3 = boto3.resource('s3',
                 region_name = "us-west-2",
                 aws_access_key_id = KEY,
                 aws_secret_access_key = SECRET)

In [ ]:
sampleDbBucket =  s3.Bucket("udacity-dend")
for bucket in sampleDbBucket.objects.filter(Prefix = 'log_data').limit(5):
    print(bucket)

In [ ]:
sampleDbBucket =  s3.Bucket("udacity-dend")
for bucket in sampleDbBucket.objects.filter(Prefix = 'song_data').limit(5):
    print(bucket)

# CLUSTER

In [6]:
redshift = boto3.client('redshift',
                  region_name='us-west-2',
                  aws_access_key_id = KEY,
                  aws_secret_access_key = SECRET)

In [7]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DB_NAME,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

In [9]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,sparkifydwh
5,Endpoint,"{'Address': 'dwhcluster.ct73jnunmupq.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-cf60f9b7
7,NumberOfNodes,2


In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

In [10]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'dwhuser',
  'DBName': 'sparkifydwh',
  'Endpoint': {'Address': 'dwhcluster.ct73jnunmupq.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2020, 3, 7, 14, 12, 21, 468000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-42298c19',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-cf60f9b7',
  'AvailabilityZone': 'us-west-2d',
  'PreferredMaintenanceWindow': 'wed:08:00-wed:08:30',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 2,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouting': False

In [11]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,deleting
3,MasterUsername,dwhuser
4,DBName,sparkifydwh
5,Endpoint,"{'Address': 'dwhcluster.ct73jnunmupq.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-cf60f9b7
7,NumberOfNodes,2
